# addition prediction problem & encoder-decoder lstm

in their 2015 [paper](https://arxiv.org/pdf/1410.4615.pdf "https://arxiv.org/pdf/1410.4615.pdf"), Wojciech Zaremba and Ilya Sutskever showed that LSTM encoder-decoder models were capable of calculating the output of small programs--adding together two numbers of up to nine digits in length.

even more impressive, the model was reading the character representations of the symbols and digits. there is nothing programatically to indicate to the model what operations are being represented--it learns this during training:

>*It is important to emphasize that the LSTM reads the entire input one character at a time and produces the output one character at a time. The characters are initially meaningless from the model’s
perspective;  for instance, the model does not know that “+” means addition or that
6
is followed
by
7
.  In fact, scrambling the input characters (e.g., replacing “a” with “q”, “b” with “w”, etc.,) has
no effect on the model’s ability to solve this problem*
>
> Wojciech Zaremba and Ilya Sutskever, Learning to Execute

as an example of how this works, the model might take in the sequence representing

12 + 4 = 16

represented in the following vectors:

`['1','2','+','5']`

`['1','7']`

the digits and symbols are just characters; they have no fucntional meaning. the model learns the relationships during training.


## sequence-to-sequence (seq2seq) model

there are a couple of key characteristics to note in this problem. first, the order matters--shuffling the order of the characters would make any relationship impossible to deduce.

second, the input and output can vary, making this problem more challenging than a one-to-one or many-to-one sequence prediction problem.

because the data is an ordered sequence of variable input and output length, this problem requires a many-to-many modeling approach, otherwise known as __sequence to sequence__.